In [1]:
import polars as pl
import pandas as pd

## 1112 higest grade for each student **

In [3]:

data = [[2, 2, 95], [2, 3, 95], [1, 1, 90], [1, 2, 99], [3, 1, 80], [3, 2, 75], [3, 3, 82]]
enrollments = pl.from_pandas(pd.DataFrame(data, columns=['student_id', 'course_id', 'grade']).astype({'student_id':'Int64', 'course_id':'Int64', 'grade':'Int64'}))
enrollments

student_id,course_id,grade
i64,i64,i64
2,2,95
2,3,95
1,1,90
1,2,99
3,1,80
3,2,75
3,3,82


In [5]:
(
    enrollments
    .with_columns(pl.col("grade").max().over("student_id").name.suffix("_max"))
    .filter(pl.col("grade")==pl.col("grade_max"))
    .drop("grade_max")
    .with_columns(pl.col("course_id").min().over("student_id"))
    .unique()
)

student_id,course_id,grade
i64,i64,i64
3,3,82
1,2,99
2,2,95


## 1607 sellers with no sales *

In [3]:
data = [[101, 'Alice'], [102, 'Bob'], [103, 'Charlie']]
customer = pl.from_pandas(pd.DataFrame(data, columns=['customer_id', 'customer_name']).astype({'customer_id':'Int64', 'customer_name':'object'}))
data = [[1, '2020-03-01', 1500, 101, 1], [2, '2020-05-25', 2400, 102, 2], [3, '2019-05-25', 800, 101, 3], [4, '2020-09-13', 1000, 103, 2], [5, '2019-02-11', 700, 101, 2]]
orders = pl.from_pandas(pd.DataFrame(data, columns=['order_id', 'sale_date', 'order_cost', 'customer_id', 'seller_id']).astype({'order_id':'Int64', 'sale_date':'datetime64[ns]', 'order_cost':'Int64', 'customer_id':'Int64', 'seller_id':'Int64'}))
data = [[1, 'Daniel'], [2, 'Elizabeth'], [3, 'Frank']]
seller = pl.from_pandas(pd.DataFrame(data, columns=['seller_id', 'seller_name']).astype({'seller_id':'Int64', 'seller_name':'object'}))

In [5]:
customer, orders, seller

(shape: (3, 2)
 ┌─────────────┬───────────────┐
 │ customer_id ┆ customer_name │
 │ ---         ┆ ---           │
 │ i64         ┆ str           │
 ╞═════════════╪═══════════════╡
 │ 101         ┆ Alice         │
 │ 102         ┆ Bob           │
 │ 103         ┆ Charlie       │
 └─────────────┴───────────────┘,
 shape: (5, 5)
 ┌──────────┬─────────────────────┬────────────┬─────────────┬───────────┐
 │ order_id ┆ sale_date           ┆ order_cost ┆ customer_id ┆ seller_id │
 │ ---      ┆ ---                 ┆ ---        ┆ ---         ┆ ---       │
 │ i64      ┆ datetime[ns]        ┆ i64        ┆ i64         ┆ i64       │
 ╞══════════╪═════════════════════╪════════════╪═════════════╪═══════════╡
 │ 1        ┆ 2020-03-01 00:00:00 ┆ 1500       ┆ 101         ┆ 1         │
 │ 2        ┆ 2020-05-25 00:00:00 ┆ 2400       ┆ 102         ┆ 2         │
 │ 3        ┆ 2019-05-25 00:00:00 ┆ 800        ┆ 101         ┆ 3         │
 │ 4        ┆ 2020-09-13 00:00:00 ┆ 1000       ┆ 103         ┆ 2        

In [16]:
with_sale = orders.filter(pl.col("sale_date").dt.year()==2020).select("seller_id").unique()
seller.filter(~pl.col("seller_id").is_in(with_sale))

seller_id,seller_name
i64,str
3,"""Frank"""
